# Sessioon 1 – Vestluse alglaadimine (Foundry Local)

See märkmik alglaadib Foundry Locali, laadib alla eelistatud mudeli aliase ning teostab nii standardse kui ka voogedastusega vestluse lõpetamise.


# Stsenaarium
Selles sessioonis tutvustatakse absoluutset miinimumi, et saada kohalik väike keelemudel vastama Foundry Locali kaudu. Te teete järgmist:
- Paigaldate SDK / kliendi sõltuvused.
- Initsialiseerite Foundry Locali halduri valitud aliase jaoks (vaikimisi: `phi-3.5-mini`).
- Rakendate kaitsemonkey‑patch'i, et taluda valikulisi välju mudeli metaandmetes.
- Saadate standardse vestluse lõpetamise päringu.
- Edastate vastuse token‑tokeni kaupa.

Eesmärk on valideerida teie kohalik tööaeg ja võrgutee enne RAG-i, marsruutimist või agentide kasutamist.


### Selgitus: Sõltuvuste paigaldamine
Paigaldab Python'i paketid, mis on vajalikud selle minimaalse vestlusvoo jaoks:
- `foundry-local-sdk`: Kohalike mudelite ja teenuste elutsükli haldamine.
- `openai`: Mugav kliendi abstraktsioon vestluste lõpetamiseks.
- `rich`: Ilus väljund selgema märkmiku kuvamiseks.

Korduv käivitamine on ohutu (idempotentne). Jäta vahele, kui sinu keskkonnas need juba olemas on.


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### Selgitus: Põhiimpordid
Toob sisse moodulid, mida kasutatakse kogu märkmikus:
- `FoundryLocalManager`, et suhelda kohaliku mudeli käitusajaga.
- `OpenAI` klient, et saaksime kasutada tuttavat vestluse lõpetamise API-d.
- `rich.print` stiilse väljundi jaoks.

Siin ei toimu ühtegi võrguühenduse kõnet—see lihtsalt valmistab nimede ruumi ette.


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### Selgitus: Halduse Initsialiseerimine ja Metaandmete Parandus
Initsialiseerib `FoundryLocalManager` valitud aliasi jaoks ja rakendab kaitsvat monkey-patch'i, et sujuvalt käsitleda teenuse vastuseid, kus `promptTemplate` võib olla `null`.

Peamised tulemused:
- Kinnitab teenuse oleku ja lõpp-punkti.
- Loetleb vahemällu salvestatud mudelid (kontrollib kohalikku salvestust).
- Lahendab aliasi konkreetse mudeli ID (kasutatakse hilisemates vestluskõnedes).

Kui kohtate valideerimisprobleeme teenuse metaandmetes, näitab see muster, kuidas neid puhastada ilma SDK-d hargnemata.


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### Selgitus: Põhiline vestluse lõpetamine
Loob `OpenAI`-ga ühilduva kliendi, mis suunatakse kohalikule Foundry lõpp-punktile, ja teostab ühe voogedastuseta vestluse lõpetamise. Keskenduge siin:
- Veenduge, et mudel vastab ilma vigadeta.
- Kontrollige latentsust / väljundi vormingut.
- Hoidke `max_tokens` tagasihoidlik, et säästa ressursse.

Kui see ebaõnnestub, kontrollige uuesti, kas Foundry Local teenus töötab ja alias lahendub õigesti.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### Selgitus: Vestluse voogedastusega lõpetamine
Näitab tokenite voogedastust, et parandada tajutavat latentsust ja interaktiivset kasutajakogemust. Tsükkel prindib järk-järgult saabuvad deltasid:
- Kasulik vestlusliidestes, kus varajane osaline väljund on oluline.
- Võimaldab mõõta tokenite läbilaskevõimet võrreldes täieliku lõpetamise latentsusega.

Seda mustrit saab kohandada tokenite kogumiseks, edenemisvidina uuendamiseks või genereerimise pooleli jätmiseks.


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
